# Homework: 使用 GPTQ 算法量化 OPT-6.7B 模型

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_name_or_path = "facebook/opt-6.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="wikitext2",
     desc_act=False,
)

quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

/root/miniconda3/envs/LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension not installed.
CUDA extension not installed.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/miniconda3/envs/LLM/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Quantizing model.decoder.layers blocks : 100%|██████████| 32/32 [11:25<00:00, 21.42s/it]
/root/miniconda3/envs/LLM/lib/python3.11/site-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantize

In [4]:
# 保存量化后的模型
quant_model.save_pretrained("models/opt-6.7b-gptq")

**试试看量化后的模型，生成效果怎么样**

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to see you're still around.
I'm still around, just not as active as I used to be.


In [6]:
text = "从前，有一只霸王龙"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(out[0], skip_special_tokens=True))

从前，有一只霸王龙的结果,他们的结果是从前的结果的结果的结果的结果的结果的结果的结果的结果的结果的结果的结果的结果的结果的结果的结�


**看来只对英语效果还可以，中文就算了把，毕竟模型和训练数据主要还是英文为主**